## Comparaison d'algorithmes de Deep RL

Dans la pratique de nombreux problèmes font intervenir une quantité gigantesques d'états possibles. Une diversité d'états rendent l'apprentissage de la table Q ou V particulièrement complexe, voir impossible pour une échelle de temps humaine (vis à vis des capacités informatiques actuelles). 

Des prémices de ce problème peuvent être observés en comparant le temps d'apprentissage de Q pour l'environnement "Frozen Lake 4x4" et "Fronze Lake 8x8". Dans la première situation l'entraînement (de Q) ne demandait que quelques secondes et convergeait vers la solution optimale. Dans le second cas il a fallu plus de 30 minutes d'entraînement (sur le même ordinateur) pour atteindre la solution optimale, cela alors que le nombre d'état n'a été multiplé que par 4 ! (De 16 à 64).


Une approche consiste à remplacer la table Q par un réseau de neurones profond (d'où le nom). On considère alors les décisions du réseaux étant donné un état comme une politique et à utiliser le fait que l'opérateur des équations de Bellman optimales est une contraction pour assurer la convergence de notre algo. (On compare les choix de l'algorithme à ceux qu'il aurait fait en suivant sa propre politique gourmande pour l'entraîner).

Ce notebook illustre les performances de deux algorithmes de Deep RL : Reinforce et DQN.

In [1]:
import gym 
import torch

from tool_functions import evaluate_drl_agent

/Users/erwan/miniconda3/envs/torch-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_id = "CartPole-v1"
env = gym.make(env_id)

state_space, s_size = env.observation_space, env.observation_space.shape[0]
print("Il y a", state_space, "états possibles")

action_space, a_size = env.action_space, env.action_space.n
print("Il y a", action_space, "actions possibles")

# Environnement d'évaluation
eval_env = gym.make(env_id)

Il y a Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32) états possibles
Il y a Discrete(2) actions possibles


Beaucoup trop d'états possibles pour initialiser une table Q -> l'utilisation des algorithmes avec les tables est trop complexe pour 
donner des bonnes performances ici.

In [3]:
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 50,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

### Algorithme Reinforce 

In [4]:
from Deep_RL.reinforce import Policy, reinforce

In [5]:
reinforce_policy =  Policy(
    game_name=cartpole_hyperparameters['env_id'],
    s_size=cartpole_hyperparameters['state_space'],
    a_size=cartpole_hyperparameters['action_space'],
    h_size=cartpole_hyperparameters['h_size']   
)

reinforce_optimizer = torch.optim.Adam(reinforce_policy.parameters(), lr=cartpole_hyperparameters['lr'])

# Entraînement
policy, scores = reinforce(reinforce_policy,
                   reinforce_optimizer,
                   cartpole_hyperparameters["n_training_episodes"], 
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"], 
                   env=env,
                   check_every=10)

Episode 10	Average Score: 18.10, Standard Deviation Score: 5.41
Métrique : 12.69
	 Nouveau meilleur score! Sauvegarde des poids du modèle à /Users/erwan/Programmes/M2DS RL/RL M2DS/best_models/reinforce_MC_CartPole-v1_12.69
Terminé !


In [6]:
mean_rwd, std_rwd = evaluate_drl_agent(
    eval_env, 
    max_steps=1000, 
    n_eval_episodes=100,
    model=policy)

print('Mean reward: {}, Std : {}'.format(mean_rwd, std_rwd))

100%|██████████| 100/100 [00:01<00:00, 89.05it/s]

Mean reward: 34.08, Std : 19.068654907989707


### Algorithme DQN

On utilise l'implémentation de Stable Baseline 3

In [7]:
from stable_baselines3 import DQN

In [11]:
dqn_model = DQN("MlpPolicy", env, verbose=1)

dqn_model.learn(total_timesteps=cartpole_hyperparameters['n_training_episodes'])

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
mean_rwd, std_rwd = evaluate_drl_agent(
    eval_env, 
    max_steps=1000, 
    n_eval_episodes=100,
    model=dqn_model)

print('Mean reward: {}, Std : {}'.format(mean_rwd, std_rwd))

100%|██████████| 100/100 [00:00<00:00, 329.36it/s]

Mean reward: 11.06, Std : 1.4684685900624503
